In [35]:
import ee
import folium
import geehydro

In [11]:
ee.Authenticate()

Enter verification code: 4/1AVHEtk4FLQ3cP4ktWhbhKYAEcKWx3q5dlmnz_90iCLNEH91T9SZ0AwCnAqc

Successfully saved authorization token.


In [13]:
ee.Initialize()

### Creating an Interactive Map

- Creating a folium map with  OpenStreetMap basemap.
- Additional basemaps can be added using the Map.setOptions() function.
- The optional basemaps can be: ROADMAP, SATELLITE, HYBRID, TERRAIN, or ESRI.

In [14]:
Map = folium.Map(location=[14.09, -59.9358], zoom_start=3) #Brazil latitude, longitude
Map.setOptions('HYBRID')

### Calculating NDVI

1. We will load countries data but for this project we will filter it to only include Brazil.
2. Then we will load two 5-year Landsat 7 composites for the years 1999-2003 and 2008-2012.
3. Then we'll compute the Normalized Difference Vegetation Index (NDVI) for both composites using: a. NDVI formula, b. normalized difference landsat2008 function from earth engine.
4. Then we'll compute the multi-band difference image between the two composites
5. Finally, we'll adds it as a top layer to the map clipping to the boundaries of Brazil.

In [15]:
#Load countries data
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Brazil = countries.filter(ee.Filter.eq("country_na","Brazil"));

# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Computing for NDVI using the formula
ndvi1999 = landsat1999.select('B4').subtract(landsat1999.select('B3')) \
               .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI via Earth Engine's normalized difference function
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

# Computing for the multi-band difference image
diff = landsat2008.subtract(landsat1999)

# Adding the computed difference on top of our basemap
Map.addLayer(diff.clip(Brazil),
             {'bands': ['B4', 'B3', 'B2'], 'min': -32, 'max': 32},
             'difference')

### Displaying our Output Map

In [31]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

### Interpretation

The resulting multi-band difference image shows the change in reflectance values between the two Landsat 7 composites for the years 1999-2003 and 2008-2012. The colors in the image represent the magnitude and direction of change in reflectance values for each pixel. Positive values (displayed in red) indicate an increase in reflectance, while negative values (displayed in blue) indicate a decrease. The magnitude of change is represented by the brightness of the color, with brighter colors indicating larger changes.

To interpret the image, you would need to have some knowledge of what the different bands represent and how changes in their reflectance values relate to changes on the ground. For example, an increase in NDVI (represented by an increase in the red band) could indicate an increase in vegetation growth, while a decrease could indicate a loss of vegetation. However, without more information about the specific location and context, it is difficult to draw any definitive conclusions from the image alone.